#do記法  
複数のモナド値を糊付けするもの
```
ghci> Just 3 >>= (\x -> Just (show x ++ "!"))
Just "3!"
```
上のコードを実行すると、ラムダ式の中のxに3が入る。この式の中では、3はモナド値ではなく、通常の値として扱える。
このラムダ式の中にもう1つ >>= があったら？
```
ghci> Just 3 >>= (\x -> Just "!" >>= (\y -> Just(show x ++ "!")))
Just "3!"
```
外側のラムダ式の中では、Just "!" をラムダ式 \y -> Just(show x ++ "!") に食わせている。このラムダ式では、 y は "!" になる。また、 x は、外側のラムダ式が実行されたときに 3 が入ったままになる。
好きな箇所を失敗で置き換えられる。
```
ghci> Nothing >>= (x -> Just "!" >>= (\y -> Just (show x ++ y)))
Nothing
ghci> Just 3 >>= (x -> Nothing >>= (\y -> Just (show x ++ y)))
Nothing
ghci> Just 3 >>= (x -> Just "!" >>= (\y -> Nothing))
Nothing
```
この式をスクリプト風に書き直してみる
```
foo :: Maybe String
foo = Just 3 >>= (\x ->
      Just "!" >>= (\y ->
      Just (show x ++ y)))
```
do記法を使えば、このようにも書ける。
```
foo :: Maybe String
foo = do
    x <- Just 3
    y <- Just "!"
    Just (show x ++ y)
```
次のコードを見てみよう。
```
ghci> Just 9 >>= (\x -> Just (x > 8))
Just True
```
これをdo記法で書き直すと
```
marySue :: Maybe Bool
marySue = do
    x <- Just 9
    Just (x > 8)
```
###いくつかの具体例
-- リストモナドの例
```
listExample :: [Int]
listExample = do
    x <- [1, 2, 3]
    y <- [4, 5]
    return (x + y)

-- 出力: [5, 6, 6, 7, 7, 8]
```
-- 上記の listExample の do 記法を展開すると
```
listExample :: [Int]
listExample = [1, 2, 3] >>= \x ->
              [4, 5] >>= \y ->
              return (x + y)
```
-- Maybe モナドの例
```
maybeExample :: Maybe Int
maybeExample = do
    x <- Just 10
    y <- Just 20
    return (x + y)

-- 出力: Just 30
```
-- IO モナドの例
```
ioExample :: IO ()
ioExample = do
    putStrLn "What's your name?"
    name <- getLine
    putStrLn ("Hello, " ++ name ++ "!")

-- 実行すると、ユーザーからの入力に基づいて挨拶が表示される。
```
-- 複数の計算を組み合わせる例
```
combinedExample :: IO ()
combinedExample = do
    putStrLn "Enter a number:"
    num1 <- readLn :: IO Int
    putStrLn "Enter another number:"
    num2 <- readLn :: IO Int
    let sum = num1 + num2
    putStrLn ("The sum of the numbers is: " ++ show sum)

-- 実行すると、2つの数値を入力して、その合計が表示される。
```





#リストモナド
リストモナドは、リストの各要素に対する計算を連鎖的に行うための便利なツール

リストモナドの操作
1. return の定義  
リストモナドの return は、リストの単一の要素を持つリストを作成
```
return :: a -> [a]
return x = [x]
```
2. 　>>= (バインド) の定義  
リストモナドのバインド演算子 >>= は、リスト内の各要素に対して関数を適用し、その結果をリストとして結合する。  
　>>= を使えばリストをいくつでも連結して非決定性を伝播させることができる。
```
(>>=) :: [a] -> (a -> [b]) -> [b]
xs >>= f = concat (map f xs)
```
-- リストモナドの例
```
main :: IO ()
main = do
    let xs = [1, 2, 3]
    -- リストの各要素に2を掛け、その結果をリストとして結合
    let doubled = xs >>= \x -> [x * 2]
    print doubled  -- [2, 4, 6]
    
    -- 複数の計算を組み合わせる例
    let result = xs >>= \x -> [x * 2] >>= \y -> [y + 1]
    print result  -- [3, 5, 7]
```

-- do記法を使用したリストモナドの例
```
main :: IO ()
main = do
    let xs = [1, 2, 3]
    let result = do
        x <- xs
        let y = x * 2
        return (y + 1)
    print result  -- [3, 5, 7]
```
